# Homework nr. 2 - data cleaning (deadline 8/11/2018)

In short, the main task is to clean The Metropolitan Museum of Art Open Access dataset.
  
> The instructions are not given in details: It is up to you to come up with ideas on how to fulfill the particular tasks as best you can. ;)

## What are you supposed to do:

  1. Download the dataset MetObjects.csv from the repository https://github.com/metmuseum/openaccess/.
  2. Check consistency of at least three features where you expect problems (include "Object Name" feature).
  3. Select some features where you expect integrity problems (describe your choice) and check integrity of those features.
  4. Convert at least five features to a proper data type. Choose at least one numeric, one categorical and one datetime.
  5. Find some outliers (describe your selection criteria).
  6. Detect missing data in at least three features, convert them to a proper representation (if they are already not), and impute missing values in at least one feature.

**If you do all this properly, you will obtain 6 points**

To earn **extra two points** you can do some of these:
  * Focus more precisely on cleaning of the "Medium" feature. Such if you like to use it in KNN based algorithms later.
  * Focus on the extraction of physical dimensions of each item (width, depth and height in centimeters) from the "Dimensions" feature.

## Comments

  * Please follow the instructions from https://courses.fit.cvut.cz/MI-PDD/homeworks/index.html.
  * If the reviewing teacher is not satisfied, he can give you another chance to rework your homework and to obtain more points.

In [33]:
import numpy as np
import pandas as pd
import sklearn as skit
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors
from scipy.stats import chi2_contingency
from IPython.display import display, Markdown, Latex

import seaborn as sns
import re

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [34]:
df = pd.read_csv('openaccess/MetObjects.csv', low_memory=False)

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 472127 entries, 0 to 472126
Data columns (total 43 columns):
Object Number              472127 non-null object
Is Highlight               472127 non-null bool
Is Public Domain           472127 non-null bool
Object ID                  472127 non-null int64
Department                 472127 non-null object
Object Name                468155 non-null object
Title                      441060 non-null object
Culture                    192200 non-null object
Period                     73147 non-null object
Dynasty                    23304 non-null object
Reign                      11212 non-null object
Portfolio                  21840 non-null object
Artist Role                281379 non-null object
Artist Prefix              96627 non-null object
Artist Display Name        282598 non-null object
Artist Display Bio         235113 non-null object
Artist Suffix              10742 non-null object
Artist Alpha Sort          282569 non-null object


* All of Object ID are different is good set is as row index.

In [36]:
df.index = df['Object ID']

## Consistency check and repair
### Check consistency of "Object Name" attribute:

In [37]:
tmp = df['Object Name'].str.strip().str.len()
display(tmp.describe())
display(list(df.loc[tmp.nlargest(100).index]['Object Name']))
display(df.loc[tmp.nsmallest(100).index]['Object Name'])
display(list(df['Object Name'].unique()))

count    468155.000000
mean         10.244402
std           8.405991
min           0.000000
25%           5.000000
50%           7.000000
75%          12.000000
max          80.000000
Name: Object Name, dtype: float64

['Double-barreled flintlock shotgun with exchangeable percussion locks and barrels',
 'Nineteen ornamental fittings from a horse harness and an ornamental saddle plate',
 'Blade and for a hatchet (Nata) and mounting for a dagger (Chiisagatana-goshirae)',
 'Set of Pommel (Fuchigashira), Sword grip ornaments (Menuki), and Cord (Kurikata)',
 'Top two lames from a right thigh defense (Cuisse), possibly from armor for a boy',
 'Manuscript (Du Pré, Galliot (fl.1506 -1561) . [Miscelánea] [Manuscrito] . S.XVI)',
 'Plate 15. Descent into Hell; Christ carrying an episcopal cross, entering tPrint',
 'Pall, sheet, Wah, rather coarse, light weight, weft fringe, torn ends, very worn',
 'String of 16 spherical, 1 barrel and 1 cylindrical beads and 2 hippo head amulet',
 'Cone, round, Padiamunnebnesuttawy, chancellor, Amun, lector, priest Beniuutehhor',
 'Scarab, Menkheperra, nsw bit, royal title, mry imn, beloved of Amun, Thutmose II',
 'Cone, Ptahmose, circular impression, high priest of Amun, vizie

Object ID
49198     \r\n
267878    \r\n
378194    \r\n
479408    \r\n
24857       Ax
25577       Ax
32944       Ax
37759       Ax
38627       Ax
38628       Ax
38629       Ax
38630       Ax
38631       Ax
44826       Ax
50575       Ax
50576       Ax
50577       Ax
50594       Ax
50617       Ax
50618       Ax
50622       Ax
50624       Ax
50625       Ax
50626       Ax
50627       Ax
50629       Ax
50631       Ax
50632       Ax
50633       Ax
50634       Ax
          ... 
248056      As
248060      As
248062      As
254565      As
256803      As
500526      Ūd
500580      Po
500581      Po
500582      Lo
500583      Lo
500586      Lo
500587      Lo
500588      Po
500589      Bo
500590      Po
500606      Gu
500607      Gu
500608      Gu
500996      Ūd
501000      Ūd
501092      Ty
503203      Ūd
503211      So
503339      Em
503345      Ūd
503765      Se
504390      Ūd
504702      Yu
505449      Lo
546958      Ax
Name: Object Name, Length: 100, dtype: object

['Coin',
 'Peso',
 'Centavos',
 'Pesos',
 'Bust',
 'Clock',
 'Vase',
 'Side Chair',
 'Figure',
 'Advertisement',
 'Ale glass',
 'Andiron',
 'Apothecary jar',
 'Statuette',
 'Frame',
 'Architectural elements',
 'Loggia elements',
 'Bedroom',
 'Argand lamp',
 'Lamp shade',
 'Armchair',
 'Banister-back armchair',
 'Drawing',
 'Slat-back armchair',
 'Spindle-back armchair',
 'Teapot',
 'Rocking chair',
 'Reclining armchair',
 'Folding armchair',
 'Side chair',
 'Revolving armchair',
 'Furniture hardware',
 'Astral lamp',
 'Window',
 'Aviary',
 "Baby's bottle",
 'Badge',
 'Baking dish',
 'Balcony',
 'Ball',
 'Wall panel',
 'Baluster',
 'Banjo clock',
 'Bank',
 'Baptismal basin',
 'Baptismal bowl',
 'Baptismal certificate',
 'Bottle',
 'Mug',
 'Keg',
 'Basin',
 'Wash basin',
 'Basin Stand',
 'Basin stand',
 'Basket',
 'Door',
 'Oil cruet',
 'Beaker',
 'Pulque beaker',
 'Tumbler',
 'Bean pot',
 'Folding bedstead',
 'Bedstead',
 'Bell',
 'Bell pull',
 'Bellows',
 'Bench',
 'Sleigh seat',
 'Pai

* "Object Name" have a lot of name with \n or \r\n characters.
* In most of the cases I came across, these characters are instead of spaces.
* Second problem in data what I find is ? or (?). 
* This can be used as a result of some uncertainty about the "Object name", 
I will leave this information in data, but I will make the embarrassment consistent with char (?).
* After repair \r\n there is some empty char will replace with nan.
* There are some two char name which can be bad, but there is not documetation for data. I will not replace it.
* Repair consistency of "Object Name" below

In [38]:
q_mark2 = re.compile(r'[?]+')
end_line1 = re.compile(r'\r\n+')
end_line2 = re.compile(r'\n+')
spaces = re.compile(r'[ ]+')
df['Object Name'] = df['Object Name'].str.replace("\(\?\)", '?').str.replace(q_mark2, '(?)')
df['Object Name'] = df['Object Name'].str.replace(end_line1, '\n').str.replace(end_line2, ' ')
df['Object Name'] = df['Object Name'].str.replace(spaces, ' ').str.strip()
df['Object Name'].replace([''], np.nan, inplace=True)

### Next check attribute for consistency is "Dynasty". Here I indicated inconsistencies in the slashes used:

In [39]:
display(df['Dynasty'].unique())

array([nan, 'Dynasty 8–11', 'Dynasty 12', 'early Dynasty 18', 'Dynasty 1',
       'Dynasty 18, early', 'Dynasty 6', 'Dynasty 5', 'Dynasty 4–5',
       'Dynasty 5–6', 'Dynasty 4–mid 5', 'Dynasty 4, mid–5th Dynasty',
       'Dynasty 4', 'Dynasty 3', 'Dynasty 12–13', 'Dynasty 6, first half',
       'Dynasty 6–8', 'Dynasty 6-8', 'Dynasty 4-6', 'early Dynasty 3',
       'Dynasty 11-12', 'Dynasty 18', 'Dynasty 27', 'Dynasty 12–17',
       'Dynasty 17–Early Dynasty 18', 'Dynasty 17–18', 'mid-Dynasty 18',
       'late Dynasty 12–early 13', 'late Dynasty 12–early  Dynasty 13',
       'Dynasty 12, late - early 13', 'Dynasty 13', 'Dynasty 11',
       'Dynasty 12, late–early 13',
       'Dynasty 12, late-13 up to 1700 B. C.', 'Dynasty 13–17',
       'late Dynasty 12–early Dynasty 13',
       'Dynasty 12, late–13 up to 1700 B. C.', 'Dynasty 12, late–13',
       'Dynasty 9 or later', 'Dynasty 9–10', 'Dynasty 9–11', 'Dynasty 17',
       'Second Intermediate Period', 'Dynasty 12, late - 13 up to 1700'

* Especially 
    * 'Dynasty 12, late - early 13'
    * 'Dynasty 12, late–early 13'
    * 'early dynasty 4–5'
    * 'Dynasty 29'
* Word dynasty will be remove, because is in column name
* Convert ? -> (?)
* Conver string to lower case
* Remove th, st, etc. after numbers
* Repair this consistency problem below

In [40]:
q_mark2 = re.compile(r'[?]+')
end_line1 = re.compile(r'\r\n+')
end_line2 = re.compile(r'\n+')
spaces = re.compile(r'[ ]+')
number = df['Dynasty'].nunique()
th = re.compile(r"(?<=\d)(st|nd|rd|th)\b")

df['Dynasty'] = df['Dynasty'].str.lower().str.replace("dynasty", '').str.replace("dyn", '')
df['Dynasty'] = df['Dynasty'].str.replace("\(\?\)", ' ? ').str.replace(q_mark2, ' (?) ')
df['Dynasty'] = df['Dynasty'].str.replace(end_line1, '\n').str.replace(end_line2, ' ')
df['Dynasty'] = df['Dynasty'].str.replace(spaces, ' ').str.strip()
df['Dynasty'] = df['Dynasty'].str.replace("–", '-').str.replace("- ", '-').str.replace(' -', '-').str.strip()
df['Dynasty'] = df['Dynasty'].str.replace(th, '')

display(Markdown("After repair consistency of values the number of unique decrease from \
                 " + str(number) + " to " +  str(df['Dynasty'].nunique())))

After repair consistency of values the number of unique decrease from                  377 to 296

### Next check attribute for consistency is "Medium":

In [41]:
display(list(df['Medium'].unique()))

['Gold',
 'Silver',
 'Bronze or copper',
 'Nickel',
 'Pressed glass',
 'Mahogany, laminated',
 'Gold, amethysts, spessartites, tourmalines, fresh water pearls, quartzes, rock crystal, and enamel',
 'Oak, tulip poplar',
 'Earthenware, basalt',
 'Paper',
 'Brass, iron',
 'Brass',
 'Iron',
 'Bell metal',
 'Bell metal, possibly iron',
 'Iron, brass',
 'Brass, steel',
 'Wrought iron',
 'Brass and iron',
 'Bronze, glass, iron',
 'Earthenware',
 'Earthenware, lusterware',
 'Bronze',
 'Wood, gilt',
 'Wood',
 'Wood, marble',
 'Limestone, ceramic, and Fravrile glass',
 'Sheffield plate, earthenware, glass',
 'Bronze, gilt brass',
 'Gilt bronze',
 'Glass',
 'Oak',
 'Maple, ash',
 'Maple',
 'Hard maple, soft maple, ash',
 'Maple, oak',
 'Graphite on off-white wove paper',
 'Poplar, ash',
 'Maple, cottonwood',
 'Oak, spruce, maple',
 'Pine, ash, maple',
 'Hickory, pine',
 'Beech',
 'Ash',
 'Mahogany',
 'Walnut, white cedar, yellow pine',
 'Maple, birch',
 'Primary: mahogany.  Secondary: cherry (med

* Here is the inconsistency mostly in the font size of the names, the first letter is always big, so for example Iron, oak are different from Oak, iron.
* Another problem is again the characters for line breaks.
* Furthermore, there is two separators "," or "and"
* Repair this consistency problem below

In [42]:
end_line1 = re.compile(r'\r\n+')
end_line2 = re.compile(r'\n+')
spaces = re.compile(r'[ ]+')
df['Medium'] = df['Medium'].str.lower().str.replace(" and ", ',').str.replace(",,", ',')
df['Medium'] = df['Medium'].str.replace(end_line1, '\n').str.replace(end_line2, ' ')
df['Medium'] = df['Medium'].str.replace(', ', ',').str.replace(',', ', ')
df['Medium'] = df['Medium'].str.replace(spaces, ' ').str.strip()

## Features with integrity problems
1. I think that data integrity problem can be found for example for attributes "Object Begin Date" and "Object End Date". There end date should be bigger than the start date. These dates are represented by year as int64.

2. Next integrity problems could be in Artist attributes. In this attributes are values for more artists separated by "|", so when I have 3 authors separated, there should be  for each of "Role", "Nationality", "Begin Date", "End Data", etc.  exist same count of values separate with "|" too.

3. Next may be if the city is in the specific country, region ... But this is necessary check externally.

### Check "Object Begin Date" and "Object End Date" integrity:

In [43]:
display((df[(df['Object Begin Date'] > df['Object End Date'])])[['Object Name', 
                                                                 'Object Begin Date', 'Object End Date']].head(10))

number = (df[(df['Object Begin Date'] > df['Object End Date'])])[['Object Name', 
                                                                 'Object Begin Date', 'Object End Date']].shape[0]
display(Markdown("Number of row which has \"Object Begin Date\"\
                 bigger than \"Object End Date\" is " + str(number) + " of " + str(df.shape[0])))
display(Markdown("There are several ways to solve this problem. \
                 For example, by replacing \"Artist End Date\" with value from begin or set to NaN or delete this rows."))

,Object Name,Object Begin Date,Object End Date
Object ID,,,
38468,Yaksha,-45,-55
55128,Print,1861,9
107853,Dolman,1875,1800
209514,Fan,1785,1779
229879,Sample and letter,1785,1773
237874,Fragments,1785,1779
243602,Ring with scarab,0,-5
247380,Lebes gamikos,-399,-3000
249163,Neck-amphora,-450,-475


Number of row which has "Object Begin Date"                 bigger than "Object End Date" is 198 of 472127

There are several ways to solve this problem.                  For example, by replacing "Artist End Date" with value from begin or set to NaN or delete this rows.

### Check Artists integrity problem:

In [44]:
x = lambda x: len(str(x).split("|")) if x is not np.nan else 0
number = df[((df['Artist Role'].apply(x) == df['Artist Display Name'].apply(x)) & 
       (df['Artist Display Bio'].apply(x) == df['Artist Alpha Sort'].apply(x)) &
       (df['Artist Begin Date'].apply(x) == df['Artist End Date'].apply(x)))].shape[0]

display(Markdown("Number of row where is OK the count of values in Artists atributes: " 
                 + str(number)))
display(Markdown("The value includes lines where all NaN values are present."))

Number of row where is OK the count of values in Artists atributes: 392990

The value includes lines where all NaN values are present.

## Convert features to a proper data type.
### Convert "Credit Line" to text ('Credit Line From') and year ('Credit Line Year') if it is possible:
### Convert "Credit Line Year" to data time type:


In [50]:
def split(x):
    x = str(x).split(",")[-1].strip()
    if x.isdigit(): return x
    else: return np.nan
def removeYear(x):
    y = str(x).split(",")[-1].strip()
    if y.isdigit(): return ",".join(str(x).split(",")[:-1])
    else: return x
    
df['Credit Line Year'] = df['Credit Line'].apply(split)
df['Credit Line From'] = df['Credit Line'].apply(removeYear)
df['Credit Line Year'] = pd.to_datetime(df['Credit Line Year'], format='%Y', errors='coerce')

### Convert "Department" to categorical type:

In [51]:
df['Department'] = df['Department'].astype('category')

### Convert "Metadata Date" to datetime category:

In [52]:
df['Metadata Date'] = pd.to_datetime(df['Metadata Date'], format='%m/%d/%Y %H:%M:%S %p')

In [68]:
#display(list(df['Dimensions'].unique()))
def getDim(x):
    y = x
    x = str(x).split(")")[0]
    x = str(x).split("(")[-1]
    x = str(x).replace("cm", "").strip()
    try:
        float(x.split(" ")[0])
    except:
        print(y)
    return x
temp = df['Dimensions'].apply(getDim)
display(len(list(temp.unique())))
display(list(temp.unique()))

Dimensions unavailable
Dimensions unavailable
Dimensions unavailable
Dimensions unavailable
Dimensions unavailable
Dimensions unavailable
Dimensions unavailable
Dimensions unavailable
Dimensions unavailable
Dimensions unavailable
Dimensions unavailable
Dimensions unavailable
Dimensions unavailable
Dimensions unavailable
Dimensions unavailable
Dimensions unavailable
Dimensions unavailable
Dimensions unavailable
Dimensions unavailable
Dimensions unavailable
Dimensions unavailable
Dimensions unavailable
Dimensions unavailable
Dimensions unavailable
Dimensions unavailable
Dimensions unavailable
Dimensions unavailable
Cup (a): H. 2 5/8 (6.7 cm)
Arm (a): W. 11 1/2 in. (29.2 cm)
b: Diam. 2 in. (5.1 cm)
Dimensions unavailable
Dimensions unavailable
Dimensions unavailable
Dimensions unavailable
Dimensions unavailable
Dimensions unavailable
Dimensions unavailable
Dimensions unavailable
Dimensions unavailable
Dimensions unavailable
Dimensions unavailable
Dimensions unavailable
Dimensions unavaila

L. (of blade) 11 1/4 in.  (28.6 cm); L. (of tang) 11 1/2 in. (29.2 cm); L. (exclusive of sheath) 22 3/4 in. (57.8 cm)
L. (blade to socket) 3 15/16 in.  (10.0 cm); L. (overall) 8-15/16 in. (22.7 cm)
Pistol (a); L. 10 3/4 in. (27.3 cm); L. of barrel 5 1/2 in. (14 cm); Cal. .43 in (10.9 mm); Wt. 1 lb. 3.8 oz. (561.3 g); holster (b); H. 12 3/8 in. (31.4 cm); W. 5 3/4 in. (14.6 cm); Wt. 2.6 oz. (73.7 g)
L. (blade to socket) 5 3/4 in.  (14.6 cm); L. (overall exclusive of sheath) 8 1/8 in. (20.6 cm)
Rifle (a); L. 55 3/4 in. (141.61 cm); L. of barrel 40 1/2 in. (102.87 cm); Cal. .51 in. (12.7 mm); Wt. 11 lb. 2 oz. (5046 g); bullet mould (c); L. 6 1/4 in. (15.9 cm); Wt. 5.6 oz. (158.8 g)
Powder horn (a); L. 11 in. (27.9 cm); Diam. 3 in. (7.6 cm); Wt. 7.1 oz. (201.3 g); shoulder belt (b); L. 42 in. (106.7 cm)
Breastplate (a); H. 13 1/2 in. (34.3 cm); W. 15 in. (38.1 cm); D. 7 1/4 in. (18.4 cm); Wt. 5 lb. 9 oz. (2523 g); reinforcing plate (b); Wt. 4 lb. 9 oz. (2070 g); lance-rest (c); Wt. 1 lb. 9

Ammunition belt with pouch (a); L. 30 1/2 in. (77.5 cm); Wt. 1 lb. 4.5 oz. (581.2 g); powder flask (b); H. 8 1/2 in. (21.6 cm); W. 5 in. (12.7 oz.); Wt. 13.7 oz. (388.4 g)
Ammunition belt (a); L. 29 in. (73.7 cm); Wt. 1.5 oz. (42.5 g); powder flask (b); H. 7 1/2 in. (19.1 cm); Wt. 7.4 oz. (209.8 g); cartridge case (c); H. 4 1/2 in. (11.4 cm); W. 6 1/2 in. (16.1 cm); Wt. 12.2 oz. (345.9 g); pouch (d); H. 4 in. (10.2 cm); W. 4 in. (10.2 cm); Wt. 5.7 oz. (161.6 g); pouch (e); H. 4 1/4 in. (10.8 cm); W. 4 in. (10.2 cm); Wt. 5.4 oz. (153.1 g); pouch (f); H. 6 in. (15.2 cm); W. 4 in. (10.2 cm); Wt. 5.5 oz. (155.9 g); cartridge case lining (g); H. 4 1/2 in. (11.4 cm); W. 6 1/2 in. (16.1 cm); cartridges (h–i); H. of each 3 1/4 in. (8.3 cm); Wt. of each 1 oz. (28.3 g)
Ammunition belt (a); L. 30 in. (77.5 cm); W. 6 1/2 in. (16.5 cm); Wt. 2 lb. 6.8 oz. (1100 g); powder flask (b); H. 8 1/2 in. (21.6 cm); W. 5 1/2 in. (14 cm); D. 1 1/2 in. (3.8 cm); Wt. 8.3 oz. (235.3 g)
Belt (a); L. 39 1/2 in. (10

H. (without dowel) 72 in. (182.9 cm); W. 32 1/2 in. (82.6 cm); D. 13 in. (33 cm)

H. (head only) 10 1/2 in. (26.7 cm); W. 5 3/8 in. (13.7 cm); D. 4 1/4 in. (10.8 cm)
H. (incl. base) 4 1/2 in. (11.4 cm); H. (without base) 3 1/4 in. (8.3 cm)
H. (incl. cover) 5 in. (12.7 cm); Diam. 4 3/4 in. (12 cm)
Sight (Image): 27 x 19 1/2 in. (68.6 x 49.5 cm)
Mounted: 46 x 25 in. (116.8 x 63.5 cm)
Overall (mold): H. 5 5/8 in. (14.3 cm); W. 2 3/4 in. (7 cm); D. 1 1/4 in. (3.2 cm)
Overall (impression): H. 6 5/8 in. (16.8 cm); W. 2 3/8 in. (6 cm); D.1 1/4 in. (3.2 cm)
Overall (mold): H. 4 5/16 in. (11 cm); W. 2 3/8 in. (6 cm); D. 1 3/8 in. (3.5 cm)
Overall (impression): H. 4 1/2 in. (11.4 cm); W. 2 1/2 in. (6.4 cm); D. 1 1/4 in. (3.2 cm)
H. (image, from top of throne to bottom of base) 13 1/2 in. (34.3 cm); Gr. W. 10 1/4 in. (26 cm); Gr. D. 3 1/4 in. (8.3 cm)
Tang: H. 6 in. (15.2 cm); Gr. W. 4 5/8 in. (10.8 cm); W. at bottom 4 1/4 in. (10.8 cm)
a): Head: H. 3 3/4 in. (9.4 cm);   
b): Body (lower waist do

(H. 1 3/4) x 3 1/8 in. (4.4 x 7.9 cm)
H. (with cover) 11 3/4 in. (29.8 cm)
Diam. (largest from 29.80.2–.16):  7 3/4 in. (19.7 cm)–L. (smallest) 3/4 in. (1.9 cm)
Diam. (largest from 29.80.2–.16):  7 3/4 in. (19.7 cm)–L. (smallest) 3/4 in. (1.9 cm)
Diam. (largest from 29.80.2–.16):  7 3/4 in. (19.7 cm)–L. (smallest) 3/4 in. (1.9 cm)
Diam. (largest from 29.80.2–.16):  7 3/4 in. (19.7 cm)–L. (smallest) 3/4 in. (1.9 cm)
Diam. (largest from 29.80.2–.16):  7 3/4 in. (19.7 cm)–L. (smallest) 3/4 in. (1.9 cm)
Diam. (largest from 29.80.2–.16):  7 3/4 in. (19.7 cm)–L. (smallest) 3/4 in. (1.9 cm)
Diam. (largest from 29.80.2–.16):  7 3/4 in. (19.7 cm)–L. (smallest) 3/4 in. (1.9 cm)
Diam. (largest from 29.80.2–.16):  7 3/4 in. (19.7 cm)–L. (smallest) 3/4 in. (1.9 cm)
Diam. (largest from 29.80.2–.16):  7 3/4 in. (19.7 cm)–L. (smallest) 3/4 in. (1.9 cm)
Diam. (largest from 29.80.2–.16):  7 3/4 in. (19.7 cm)–L. (smallest) 3/4 in. (1.9 cm)
Diam. (largest from 29.80.2–.16):  7 3/4 in. (19.7 cm)–L. (smalle

...

Sheet (Trimmed): 11 7/8 × 9 5/8 in. (30.1 × 24.4 cm)
Sheet (Trimmed): 6 7/16 × 8 11/16 in. (16.4 × 22 cm)
Sheet (Trimmed): 6 7/16 × 8 11/16 in. (16.4 × 22 cm)
Sheet (Trimmed): 5 11/16 × 3 11/16 in. (14.4 × 9.4 cm)
Sheet (Trimmed): 5 5/16 × 3 5/8 in. (13.5 × 9.2 cm)
Sheet (Trimmed): 15 3/4 × 10 11/16 in. (40 × 27.1 cm)
Sheet (Trimmed): 14 1/16 × 17 1/2 in. (35.7 × 44.5 cm)
Sheet (Trimmed): 11 5/8 × 10 9/16 in. (29.5 × 26.8 cm)
Sheet (Trimmed): 18 1/2 × 14 3/16 in. (47 × 36.1 cm)
Sheet (Trimmed): 14 9/16 × 19 1/8 in. (37 × 48.5 cm)
Sheet (Trimmed): 14 9/16 × 19 1/8 in. (37 × 48.5 cm)
Sheet (Trimmed): 12 5/8 × 9 1/16 in. (32.1 × 23 cm)
Sheet (Trimmed): 12 3/8 × 9 1/16 in. (31.4 × 23 cm)
Sheet (Trimmed): 18 1/16 × 13 1/16 in. (45.9 × 33.1 cm)
Sheet (Trimmed): 11 11/16 × 12 5/8 in. (29.7 × 32 cm)
Sheet (Trimmed): 10 11/16 × 12 1/2 in. (27.1 × 31.7 cm)
Sheet (Trimmed): 14 1/4 × 16 3/4 in. (36.2 × 42.6 cm)
Sheet (Trimmed): 15 9/16 × 11 1/2 in. (39.5 × 29.2 cm)
Sheet (Trimmed): 12 1/16 × 7 15/

Sheet (Trimmed): 4 3/4 × 7 7/16 in. (12 × 18.9 cm)
Sheet (Trimmed): 4 13/16 × 7 1/2 in. (12.2 × 19 cm)
Sheet (Trimmed): 9 in. × 6 9/16 in. (22.9 × 16.6 cm)
Sheet (Trimmed): 12 3/8 × 7 13/16 in. (31.4 × 19.9 cm)
Sheet (Trimmed): 6 3/4 × 8 15/16 in. (17.2 × 22.7 cm)
Sheet (Trimmed): 6 3/4 × 8 15/16 in. (17.2 × 22.7 cm)
Sheet (Trimmed): 6 7/16 × 8 15/16 in. (16.3 × 22.7 cm)
Sheet (Trimmed): 8 9/16 × 12 1/2 in. (21.8 × 31.7 cm)
Sheet (Trimmed): 8 9/16 × 12 1/8 in. (21.8 × 30.8 cm)
Sheet (Trimmed): 13 3/4 × 17 5/16 in. (34.9 × 44 cm)
Sheet (Trimmed): 13 9/16 × 17 1/16 in. (34.5 × 43.4 cm)
Sheet (Trimmed): 11 7/8 × 13 9/16 in. (30.1 × 34.4 cm)
Sheet (Trimmed): 10 1/2 × 7 11/16 in. (26.6 × 19.5 cm)
Sheet (Trimmed): 17 5/16 × 12 11/16 in. (44 × 32.3 cm)
Sheet (Trimmed): 10 3/16 × 13 7/16 in. (25.8 × 34.2 cm)
Sheet (Trimmed): 17 13/16 in. × 13 in. (45.3 × 33 cm)
Sheet (Trimmed): 8 7/16 × 11 5/16 in. (21.4 × 28.7 cm)
Sheet (Trimmed): 6 11/16 × 11 15/16 in. (17 × 30.3 cm)
Sheet (Trimmed): 12 5/16

Sheet (Trimmed): 3 9/16 × 18 3/8 in. (9.1 × 46.7 cm)
Sheet (Trimmed): 3 9/16 × 18 9/16 in. (9.1 × 47.2 cm)
Sheet (Trimmed): 18 11/16 × 12 15/16 in. (47.5 × 32.8 cm)
2 volumes : illustrations (some color) ; Height: 15 3/4 in. (40 cm)
Sheet (Trimmed): 13 in. × 18 7/16 in. (33 × 46.9 cm)
Sheet (Trimmed): 5 13/16 × 3 1/4 in. (14.7 × 8.2 cm)
Sheet (Trimmed): 5 13/16 × 3 1/4 in. (14.7 × 8.2 cm)
Sheet (Trimmed): 5 5/8 × 3 1/8 in. (14.3 × 8 cm)
Sheet (Trimmed): 4 13/16 in. × 3 in. (12.2 × 7.6 cm)
Sheet (Trimmed): 5 3/8 × 3 1/8 in. (13.7 × 7.9 cm)
Sheet (Trimmed): 12 13/16 × 9 5/8 in. (32.5 × 24.5 cm)
Sheet (Trimmed): 13 3/16 × 9 5/8 in. (33.5 × 24.5 cm)
Sheet (Trimmed): 9 5/8 × 12 3/8 in. (24.5 × 31.4 cm)
Diam. 1.7 cm (11/16 in.)
Sheet (Trimmed): 4 in. × 1 9/16 in. (10.1 × 3.9 cm)
Sheet (Trimmed): 10 13/16 × 7 1/2 in. (27.5 × 19 cm)
Image: 5 1/2 × 3 3/4 in. (14 × 9.5 cm)
Sheet (Trimmed): 8 3/16 × 5 3/8 in. (20.8 × 13.7 cm)
Image: 4 7/16 × 3 1/4 in. (11.2 × 8.2 cm)
Sheet (Trimmed): 3 7/8 × 1 9/

155371

['Dimensions unavailable',
 '1.7',
 '1.3',
 '2.9',
 '1.9',
 '5.4',
 '7 x 8.9 x 7',
 '61.9 x 37.1 x 13',
 '49.4 x 33 x 23.5',
 '101.6 x 64.8 x 87.6',
 '30.5',
 '31.4',
 '27.9 x 22.9',
 '16.7',
 '15.9',
 '78.4 x 63.5 x 35.2',
 '68.6',
 '40.6',
 '43.2',
 '29.2',
 '41.9',
 '58.4',
 '57.2 x 58.4',
 '64.1 x 54.6 x 34.3',
 '59.7',
 '47 x 56.5 x 34.3',
 '53.3 x 53.3 x 34.3',
 '78.1 x 71.1',
 '54 x 45.1',
 '45.1 x 61',
 '64.8 x 56.5 x 34.3',
 '41 x 38.7 x 20.6',
 '48.9 x 49.5 x 29.8',
 '52.7 x 49.5 x 29.8',
 '47.6 x 43.2',
 '59.1 x 59.7',
 '53.3 x 40',
 '59.1 x 46.4',
 '28.9 x 36.8 x 30.5',
 '47.6 x 44.5 x 27.3',
 '57.2 x 68.6 x 33',
 '61 x 29.2 x 40.6',
 '57.2',
 '41 x 25.4 x 37.8',
 '47 x 43.2',
 '62.2 x 51.8',
 '53.7',
 '80 x 35.6 x 59.7',
 '63.5 x 38.1 x 88.9',
 '57.5 x 27.3 x 52.1',
 '9.2',
 '7',
 '109.2',
 '210.8 x 148.6',
 '640.1 x 701',
 '44.5',
 '45.4 x 43.2 x 20',
 '86.4 x 59.7 x 48.3',
 '101.6 x 63.5 x 44.5',
 '111.8 x 61 x 38.1',
 '113 x 58.4 x 57.2',
 '121.9 x 74.9 x 42.9',
 '14 x 

In [17]:
listt = [1,2,3,4]
print(listt[:-1])
#display(list(df['Dimensions'].unique()))

[1, 2, 3]


In [49]:
display(df.info())
display(df.nunique())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 472127 entries, 1 to 816510
Data columns (total 43 columns):
Object Number              472127 non-null object
Is Highlight               472127 non-null bool
Is Public Domain           472127 non-null bool
Object ID                  472127 non-null int64
Department                 472127 non-null object
Object Name                468151 non-null object
Title                      441060 non-null object
Culture                    192200 non-null object
Period                     73147 non-null object
Dynasty                    23304 non-null object
Reign                      11212 non-null object
Portfolio                  21840 non-null object
Artist Role                281379 non-null object
Artist Prefix              96627 non-null object
Artist Display Name        282598 non-null object
Artist Display Bio         235113 non-null object
Artist Suffix              10742 non-null object
Artist Alpha Sort          282569 non-null object


None

Object Number              469461
Is Highlight                    2
Is Public Domain                2
Object ID                  472127
Department                     19
Object Name                 27555
Title                      245972
Culture                      7183
Period                       1769
Dynasty                       296
Reign                         376
Portfolio                    3181
Artist Role                  6252
Artist Prefix                5526
Artist Display Name         60778
Artist Display Bio          44814
Artist Suffix                1697
Artist Alpha Sort           60784
Artist Nationality           3806
Artist Begin Date           23697
Artist End Date             24021
Object Date                 32210
Object Begin Date            2105
Object End Date              2078
Medium                      61051
Dimensions                 253620
Credit Line                 38688
Geography Type                132
City                         2663
State         